#prepare

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings
from numpy import dot
from numpy.linalg import norm

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import pairwise_distances

warnings.filterwarnings(action='ignore')
%matplotlib inline

#data

In [3]:
data = pd.read_csv('./Books.csv', encoding = 'utf-8')
data = data.dropna(subset = ['keyword'], axis = 0).reset_index(drop = True)

In [4]:
keywords = []
for i in range(len(data)) :
    keywords.extend(data['keyword'][i].split(' '))
keywords = list(set(keywords))

#function

In [5]:
def get_sim_score(score, idx, top = 10):
  score = [(i, c) for i, c in enumerate(score[idx]) if i != idx]
  score = sorted(score, key = lambda x: x[1], reverse=True)
  score = [(id2title[i], score) for i, score in score[0:top]]
  return score

#one-hot-encoding

In [8]:
def one_hot(data) :

    one_hot = []

    for i in range(len(data)) :
        tmp = [0] * len(keywords)
        tmp_keyword = data['keyword'][i].split(' ')

        for j in range(len(tmp_keyword)) :
            idx = keywords.index(tmp_keyword[j])
            tmp[idx] = 1

        one_hot.append(tmp)

    return one_hot

In [9]:
df = pd.DataFrame(one_hot(data), columns = keywords)

In [10]:
id2title = {}
title2id =  {}

for id, tilte in enumerate(data['title']):
  id2title[id] = tilte
  title2id[tilte] = id

In [11]:
df.iloc[:, :] = df.values.astype(int)
df.head()

,인생은끝까지모르는거야,IT용어,현실육아,문재인대통령추천,인생선배,N잡,아빠와아들,운동어떻게시작하지?,영어덜트소설,덕업일치,...,월드클래스,인문육아,드라마에나온책,해외시집,제주여행,억만장자,주식초보,혼밥족,데이터관리,SNS에세이
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#similarity

In [12]:
idx = title2id['내가 틀릴 수도 있습니다 - 숲속의 현자가 전하는 마지막 인생 수업']

In [13]:
# 내적
dot_scores = np.dot(df.values, df.values.T)
get_sim_score(dot_scores, idx)

[('어른의 어휘력 - 말에 품격을 더하고 세상을 올바르게 이해하는 힘', 2),
 ('나는 나무처럼 살고 싶다 - 30년간 아픈 나무들을 돌봐 온 나무 의사 우종영이 나무에게 배운 단단한 삶의 지혜 35', 2),
 ('반야심경 마음공부 - [ 개정판 ]', 2),
 ('어른의 문해력 - 나도 쓱 읽고 싹 이해하면 바랄 게 없겠네', 2),
 ('라틴어 수업 - 지적이고 아름다운 삶을 위한', 2),
 ('유럽 도시 기행 1 - 아테네, 로마, 이스탄불, 파리 편', 2),
 ('마음의 법칙 - 사람의 마음을 사로잡는 51가지 심리학', 2),
 ('빅터 프랭클의 죽음의 수용소에서 - 죽음조차 희망으로 승화시킨 인간 존엄성의 승리', 2),
 ('서른살이 심리학에게 묻다 - 대한민국 30대를 위한 심리치유 카페', 2),
 ('거꾸로 읽는 세계사 - [ 전면개정판 ]', 2)]

In [14]:
# 피어슨 유사도
pearson_scores = df.T.corr().values
get_sim_score(pearson_scores, idx)

[('서른살이 심리학에게 묻다 - 대한민국 30대를 위한 심리치유 카페', 0.5768308356439008),
 ('스눕 - 상대를 꿰뚫어보는 힘', 0.5768308356439008),
 ('백설공주에게 죽음을', 0.5768308356439008),
 ('온전히 나답게- 인생은 느슨하게 매일은 성실하게 ', 0.4704497235365045),
 ('다산선생 지식경영법 - [ 양장 ]', 0.40778932652959554),
 ('미쳐야 미친다 - 조선 지식인의 내면 읽기', 0.40778932652959554),
 ('십자군 이야기 1', 0.40778932652959554),
 ('노인이 되지 않는 법', 0.40778932652959554),
 ('청소부 밥', 0.40778932652959554),
 ('끌리는 사람은 1%가 다르다', 0.40778932652959554)]

In [15]:
# 코사인 유사도
cosine_scores = cosine_similarity(df, df)
get_sim_score(cosine_scores, idx)

[('서른살이 심리학에게 묻다 - 대한민국 30대를 위한 심리치유 카페', 0.5773502691896258),
 ('스눕 - 상대를 꿰뚫어보는 힘', 0.5773502691896258),
 ('백설공주에게 죽음을', 0.5773502691896258),
 ('온전히 나답게- 인생은 느슨하게 매일은 성실하게 ', 0.4714045207910318),
 ('몰입의 즐거움 - [ 양장 ]', 0.4082482904638631),
 ('100만 클릭을 부르는 글쓰기 - 블로그, 포스트, SNS, 유튜브… 바로 써먹는 글쓰기 디테일', 0.4082482904638631),
 ('행복의 조건 - 하버드대학교·인간성장보고서', 0.4082482904638631),
 ('니코마코스 윤리학', 0.4082482904638631),
 ('법화경 마음공부 - 인생이 한결 홀가분해지는', 0.4082482904638631),
 ('30개 도시로 읽는 일본사 - 익숙하고 낯선 도시가 들려주는 일본의 진짜 역사 이야기', 0.4082482904638631)]

In [16]:
# 유클리드 거리
euclidean_scores = euclidean_distances(df, df)
get_sim_score(euclidean_scores, idx)

[('소년이 온다', 4.898979485566356),
 ('사피엔스 - 유인원에서 사이보그까지, 인간 역사의 대담하고 위대한 질문', 4.795831523312719),
 ('시선으로부터,', 4.69041575982343),
 ('우리가 빛의 속도로 갈 수 없다면', 4.58257569495584),
 ('아몬드', 4.58257569495584),
 ('나미야 잡화점의 기적', 4.58257569495584),
 ('여자 둘이 살고 있습니다', 4.58257569495584),
 ('천 개의 파랑', 4.47213595499958),
 ('데미안', 4.47213595499958),
 ('82년생 김지영', 4.47213595499958)]

In [17]:
# 자카르도 거리
pairwise_scores = pairwise_distances(df.values, metric='jaccard')
get_sim_score(pairwise_scores, idx)

[('당신도 느리게 나이 들 수 있습니다 - 나이가 들어도 몸의 시간은 젊게', 1.0),
 ('빅 히스토리 - 우주와 지구, 인간을 하나로 잇는 새로운 역사', 1.0),
 ('여덟 단어 - 인생을 대하는 우리의 자세', 1.0),
 ('인생의 허무를 어떻게 할 것인가', 1.0),
 ('아비투스 - 인간의 품격을 결정하는 7가지 자본', 1.0),
 ('사피엔스 - [ 양장 ]', 1.0),
 ('뉴욕 정신과 의사의 사람 도서관 - 낙인과 혐오를 넘어 이해와 공존으로', 1.0),
 ('블루프린트 - 이기적 인간은 어떻게 좋은 사회를 만드는가', 1.0),
 ('어른의 중력  - 생의 1/4 승강장에 도착한 어린 어른을 위한 심리학', 1.0),
 ('차이에 관한 생각 - 영장류학자의 눈으로 본 젠더', 1.0)]